In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from data_cleaning import *
import implementations as imp
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2


# Import data

In [2]:
y_train, x_train, ids_train = load_csv_data("../data/train.csv")
y_test, x_test, ids_test = load_csv_data("../data/test.csv")

In [3]:
x_train.shape

(250000, 30)

# Clean test data and add features

In [53]:
NUM_JETS = 4

PRI_jet_num_train = np.array([x_train[:, COLUMN_TO_DROP]]).astype(int)
print(PRI_jet_num_train)
del_x_train = np.delete(x_train, COLUMN_TO_DROP, axis=1)
print(del_x_train.shape)

replaced_x_train = replace_undefined(del_x_train, UNDEFINED_VALUE, 0.0)

norm_x_train, train_data_mean, train_data_std = mean_std_normalization(replaced_x_train)
print(norm_x_train[0][0])
print(norm_x_train.shape)

[[2 1 1 ... 1 0 0]]
(250000, 29)
0.4986371179215161
(250000, 29)


# Do the same for the test data

In [54]:
PRI_jet_num_test = np.array([x_test[:, COLUMN_TO_DROP]]).astype(int)
print(PRI_jet_num_test)
print(PRI_jet_num_test.shape)
del_x_test = np.delete(x_test, COLUMN_TO_DROP, axis=1)
print(del_x_test.shape)

replaced_x_test = replace_undefined(del_x_test, UNDEFINED_VALUE, 0.0)

norm_x_test, test_data_mean, test_data_std = mean_std_normalization(replaced_x_test, train_data_mean, train_data_mean)
print(norm_x_test[0][0])
print(norm_x_test.shape)



[[0 1 0 ... 0 1 0]]
(1, 568238)
(568238, 29)
-1.0096823731792868
(568238, 29)


# Split data based on PRI_jet_num

In [55]:
#this can be done by splitting the data into 4 arrays, but it would be harder to reasseble in the correct order then to
#simply use the masks in the training of the model

x_split_train = np.empty([4,1])
y_split_train = np.empty([4,1])
counter = 0
for val in np.nditer(PRI_jet_num_train):
    np.append(x_split_train[val],norm_x_train[counter])
    np.append(y_split_train[val],y_train[counter])
    counter = counter + 1
    
print(counter)
x_split_test = np.empty([4,1])
counter = 0
for val in np.nditer(PRI_jet_num_test):
    np.append(x_split_test[val],norm_x_test[counter])
    counter = counter + 1
    
print(counter)

250000
568238


# Train model

In [56]:
init_w = np.ones(30)
max_iters = 100
gamma = 0.000001
degree = 5
b_size = 1
ratio = 0.5
seed = 1

for jet in range(NUM_JETS):
    (tr_x, tr_y, te_x,te_y) = split_data(x_split_train[jet],y_split_train[jet],ratio, seed)

    px_tr = build_poly(tr_x,degree)
    px_te = build_poly(te_x,degree)
    
    np.shape(px_tr)
    w_list = []
    for i in range(50):
        _, w = imp.ridge_regression(tr_y,px_tr, gamma)
        w_list.append(w)
        gamma = gamma + 0.000002

    print("shape of px_tr {}".format(px_tr.shape))
    print("shape of tr_x {}".format(tr_x.shape))
    print("shape of w {}".format(w.shape))
    print("shape of init_w {}".format(init_w.shape))
    print("shape of te_x {}".format(te_x.shape))
    print("shape of px_te {}".format(px_te.shape))


    y_pred = predict_labels(w, px_te)
    print(w_list[0].shape)
    n = len(y_pred)
    correct = 0
    for j in range(len(w_list)):
        correct = 0
        y_pred = predict_labels(w_list[j], px_te)
        for i in range(n):
            if y_pred[i] == te_y[i]:
                correct = correct + 1
            ratio = correct/n
        print(j, ratio)

LinAlgError: Singular matrix

# test accuracy

In [ ]:
y_pred = predict_labels(w, px_te)
print(w_list[0].shape)
n = len(y_pred)
correct = 0
ratio = []
for j in range(len(w_list)):
    correct = 0
    y_pred = predict_labels(w_list[j], px_te)
    for i in range(n):
        if y_pred[i] == te_y[i]:
            correct = correct + 1
        ratio.append(correct/n)
    #print(j, ratio)
print(max(ratio))

In [ ]:

test_poly = build_poly(x_test,degree)

y_pred_test = predict_labels(w_list[0],test_poly)

create_csv_submission(ids_test, y_pred_test, "y_pred.csv")